In [ ]:
# %% Equation playground
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import altair as alt
from tqdm import tqdm
from IPython.display import clear_output

# Rough test on how each parameter affect semantic input
- Purpose: rapid prototype formula and parameter before actual sampling
- Target: 
    - Around 4.5 input in LF, 5.5 in HF at the end of training
    - Effect should quickly peak at 0.1M sample at 1.0, and decrease to 0.5 at EoT
- Frequency of HF and LF is a rough copy of simulated data

## Simulate sampling

In [ ]:

class SemanticExperiment:
    """Semantic experiment class for evaluating semantic equation"""
    # Mean in each condition in Strain data set (in a perfect world)
    strain = {
                "HF_HI": {"wf": 6500., "img": 6.},
                "HF_LI": {"wf": 6500., "img": 3.5},
                "LF_HI": {"wf": 400., "img": 6.},
                "LF_LI": {"wf": 400., "img": 3.5}
            }

    def __init__(self, **kwargs):

        # Load tmp cfg and data
        import sys
        sys.path.append("/home/jupyter/tf/src/")
        import meta, data_wrangling
        from importlib import reload
        reload(data_wrangling)
        self.cfg = meta.ModelConfig.from_json(
            "../../models/test_sampling_speed_2/model_config.json")
        self.data = data_wrangling.MyData()
        clear_output()
        self.semantic_params = kwargs

        # Fake data
        self.fake_epoch = np.arange(100)
        self.fake_cwfhf = np.linspace(0, 650, 100)
        self.fake_cwflf = np.concatenate((np.linspace(0, 5, 11),  np.linspace(5, 130, 90)[1:]))

        # Instantiate sampler
        self.sampler = data_wrangling.Sampling(self.cfg, self.data, debugging=True)
        self.sampler.set_semantic_parameters(**self.semantic_params)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def run_sampling(self):
        last_epoch = 0
        with tqdm(total=100) as progress:
            while self.sampler.current_epoch <= 100:

                # Progress bar
                if last_epoch != self.sampler.current_epoch:
                    progress.update(1)
                last_epoch = self.sampler.current_epoch

                # dry run sampling
                next(self.sampler.sample_generator(dryrun=True))

        self.df, self.df_mean = self._parse_sampler_results()

        clear_output()

    def _parse_sampler_results(self):

        # Convert dict to df
        df_wf = pd.DataFrame(self.sampler.debug_wf).transpose()
        df_wf.columns = "cwf_" + df_wf.columns.astype(str)
        df_wf["word"] = df_wf.index

        df_sem = pd.DataFrame(self.sampler.debug_sem).transpose()
        df_sem.columns = "sem_" + df_sem.columns.astype(str)
        df_sem["word"] = df_sem.index

        # Merge
        df = pd.merge(df_wf, df_sem, "left", on="word")

        # Subset
        df = df.loc[df.word.isin(self.data.df_strain.word)]
        df = df.melt(id_vars=['word'],
                            var_name='epoch_label', value_name='value')

        # Make new columns
        df["measure"] = df.epoch_label.str.split("_", expand=True).loc[:, 0]
        df["epoch"] = df.epoch_label.str.split("_", expand=True).loc[:, 1]
        df.pop("epoch_label")

        # Pivot
        df = df.pivot_table(index=["word", "epoch"], columns="measure").reset_index()
        df.columns = ['word', 'epoch', 'cumulative_frequency', 'semantic_input']

        # Merge strain conditions
        df = pd.merge(df, self.data.df_strain, "left", on='word')
        df['cond'] = df.frequency + '_' + df.imageability
        df.rename(columns={'wf': 'static_wf'}, inplace=True)

        # Activation
        df["activation"] = df.semantic_input.apply(self.sigmoid)

        # Mean df
        df_mean = df.pivot_table(index="epoch", columns="cond")
        df_mean["epoch"] = df_mean.index.astype(int)
        df_mean.reset_index(drop=True, inplace=True)
        df_mean.sort_values("epoch", inplace=True)

        for measure in ["semantic_input", "activation"]:
            df_mean[f"frequency_effect_{measure}"] = df_mean[measure]["HF_HI"] + df_mean[measure]["HF_LI"] - \
                df_mean[measure]["LF_HI"] - df_mean[measure]["LF_LI"]

            df_mean[f"imageability_effect_{measure}"] = df_mean[measure]["HF_HI"] + df_mean[measure]["LF_HI"] - \
                df_mean[measure]["HF_LI"] - df_mean[measure]["LF_LI"]

            df_mean[f"fxi_interaction_{measure}"] = df_mean[measure]["LF_HI"] - df_mean[measure]["LF_LI"] - \
                df_mean[measure]["HF_HI"] + df_mean[measure]["HF_LI"]


        return (df, df_mean)

    def plot_fake_input(self):

        # Effects
        hf = self.sampler.semantic_input(self.fake_cwfhf)
        lf = self.sampler.semantic_input(self.fake_cwflf)
        eff = hf-lf

        # Plot
        ax = plt.subplot()
        plt.title("Semantic input")
        ax.plot(self.fake_epoch, hf, label="HF")
        ax.plot(self.fake_epoch, lf, label="LF")
        ax.plot(self.fake_epoch, eff, label="frequency effect")
        ax.legend()
        plt.show()

    def plot_fake_wf(self):
        ax = plt.subplot()
        plt.title("Fake cumulative frequency")
        ax.plot(self.fake_epoch, self.fake_cwfhf, label="HF")
        ax.plot(self.fake_epoch, self.fake_cwflf, label="LF")
        ax.legend()
        plt.show()


    def plot_wf(self):
        """Plot the simulated cumulative frequency"""

        plot_wf = alt.Chart(self.df).mark_line().encode(
            x="epoch:Q",
            y="mean(cumulative_frequency):Q",
            color="cond:N",
        )

        return plot_wf

    def plot_strain(self, df=None):

        if df is None:
            df = self.df_mean

        strain_conds = self.strain.keys()
        fig = plt.figure(figsize=(15, 10))

        # Semantic input
        ax = fig.add_subplot(221)
        ax.title.set_text("Semantic input over epoch")
        for condition in strain_conds:
            ax.plot(df.epoch, df["semantic_input"][condition], label=condition)
        ax.legend()

        # Semantic activation
        ax = fig.add_subplot(222)
        ax.title.set_text("Semantic activation over epoch")
        for condition in strain_conds:
            ax.plot(df.epoch, df["activation"][condition], label=condition)
        ax.legend()

        # Contrasts for input
        contrasts = ["frequency_effect", "imageability_effect", "fxi_interaction"]

        ax = fig.add_subplot(223)
        ax.title.set_text("Contrasts for input")
        for contrast in contrasts:
            ax.plot(df.epoch, df[f"{contrast}_semantic_input"], label=contrast)
        ax.legend()

        # Contrasts for activation
        ax = fig.add_subplot(224)
        ax.title.set_text("Contrasts for activation")

        for contrast in contrasts:
            ax.plot(df.epoch, df[f"{contrast}_activation"], label=contrast)
        ax.legend()



In [ ]:
proto = SemanticExperiment(**{"g":1, "k":1, "h":1, "w":1})
proto.run_sampling()



In [ ]:
alt.Chart(proto.df.loc[proto.df.epoch==proto.df.epoch.max(),]).mark_circle().encode(
    x=alt.X("static_wf:Q"),
    y=alt.Y("cumulative_frequency:Q"),
    color="cond:N",
    tooltip=["word", "cond", "cumulative_frequency", "static_wf"]
)

In [ ]:
proto = SemanticExperiment(**{"g":5, "k":100, "w":5})
proto.plot_fake_input()

